In [2]:
import pandas as pd
import json
import tqdm
import io

In [3]:
# df = pd.read_json("rawdata/All_Beauty.jsonl", lines=True)
# df_meta = pd.read_json("rawdata/meta_All_Beauty.jsonl", lines=True)

files = ["All_Beauty.jsonl", "Amazon_Fashion.jsonl", "Appliances.jsonl", "Health_and_Personal_Care.jsonl"]

for i, file in enumerate(files):
    rawdata = f"rawdata/{file}"
    rawmetadata = f"rawdata/meta_{file}"
    
    if i > 0:
        df_temp = pd.read_json(rawdata, lines=True)
        df_meta_temp = pd.read_json(rawmetadata, lines=True)

        df = pd.concat([df, df_temp])
        df_meta = pd.concat([df_meta, df_meta_temp])
    else:
        df = pd.read_json(rawdata, lines=True)
        df_meta = pd.read_json(rawmetadata, lines=True)
    print(i)
    i += 1

0
1
2
3


In [4]:
df = df.reset_index()
df_meta = df_meta.reset_index()
df = df.drop(columns=['index'])
df_meta = df_meta.drop(columns=['index'])

In [5]:
#count amount of images
df['images'] = df['images'].apply(len)

In [6]:
df_merge = df.merge(df_meta, right_on="parent_asin", left_on="parent_asin", suffixes=("_data", "_meta"))

In [5]:
print(df.isna().sum())
print(df_meta.isna().sum())

rating               0
title                0
text                 0
images               0
asin                 0
parent_asin          0
user_id              0
timestamp            0
helpful_vote         0
verified_purchase    0
dtype: int64
main_category         4676
title                    0
average_rating           0
rating_number            0
features                 0
description              0
price               968103
images                   0
videos                   0
store                41431
categories               0
details                  0
parent_asin              0
bought_together    1093318
subtitle           1093313
author             1093317
dtype: int64


In [7]:
def count_words(text):
    if not isinstance(text, str):
        return 0
    return len(text.split())

df_merge['review_length'] = df_merge['text'].apply(count_words)
df_merge['title_length'] = df_merge['title_data'].apply(count_words)
df_merge = df_merge.drop(columns=['bought_together', 'price', 'subtitle', 'author'])
df_merge = df_merge.dropna(subset=['store'])


In [8]:
import datetime
df_merge = df_merge[df_merge['timestamp'] > datetime.datetime(2018, 1, 1)]

In [11]:
# %pip install transformers
# %pip install torch
# %pip install tf-keras

import torch
from transformers import pipeline
tqdm.tqdm.pandas()

# Load the classification pipeline with the specified model
pipe = pipeline("text-classification", 
				model="tabularisai/multilingual-sentiment-analysis", 
				device=0,
				batch_size=128,
				truncation=True)


# Truncate text to 512 characters to avoid model input size errors
def safe_sentiment(text):
	if not isinstance(text, str):
		return None
	# Truncate to 512 characters (approximate, as tokenization may differ)
	truncated = text[:512]
	try:
		return pipe(truncated)[0]['label']
	except Exception as e:
		print(f"Error processing text: {e}")
		return None

df_merge['sentiment'] = df_merge['text'].progress_apply(safe_sentiment)


Device set to use cuda:0
100%|██████████| 3907722/3907722 [6:29:39<00:00, 167.14it/s]  


In [ ]:
df_merge.to_pickle("df_processed_sentiment.pkl")

In [ ]:
df_merge.to_pickle("df_processed_sentiment.pkl")


In [ ]:
df_merge['sentiment'].value_counts()

sentiment
Positive         1885157
Negative         1163476
Neutral           372627
Very Positive     288672
Very Negative     197790
Name: count, dtype: int64